In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
import os

sys.path.append(os.path.abspath(".."))

from ARISA_DSML.preproc import get_raw_data
from ARISA_DSML.config import RAW_DATA_DIR, target, categorical

In [15]:
import os
from pathlib import Path
container_check = os.getenv("iscontainer")
if container_check=="y":
    config_dir = Path("/home/vscode/.config/kaggle")
    config_dir.mkdir(parents=True, exist_ok=True)

    with open(config_dir / "kaggle.json", "w") as dst:
        with open("../kaggle.json", "r") as src:
            dst.write(src.read())

In [18]:
get_raw_data()
!dir "{RAW_DATA_DIR}"

2025-05-06 18:43:04.310 | INFO     | ARISA_DSML.preproc:get_raw_data:6 - Downloading data to: /workspaces/Predict_cholesterol/ARISA_MLOps-Structure/data/raw


Dataset URL: https://www.kaggle.com/datasets/joannborkowska/cholesterol-supplementation-classification


2025-05-06 18:43:04.820 | INFO     | ARISA_DSML.preproc:get_raw_data:8 - Dataset download and extraction completed successfully


test_cholesterol.xlsx  train_choloesterol.xlsx


In [30]:
from ARISA_DSML.preproc import preprocess_df

train_path = preprocess_df(RAW_DATA_DIR / "train_choloesterol.xlsx")

In [32]:
import pandas as pd
df_train = pd.read_csv(train_path)

In [35]:
y_train = df_train.pop(target)
X_train = df_train

In [37]:
from ARISA_DSML.train import run_hyperopt

best_params_path = run_hyperopt(X_train, y_train)

[I 2025-05-06 19:41:42,650] A new study created in memory with name: no-name-89f1d8df-9a1a-4330-a033-66ecaeb3b131


[I 2025-05-06 19:41:42,896] Trial 0 finished with value: 0.16442104830261187 and parameters: {'depth': 9, 'learning_rate': 0.17264972139269638, 'iterations': 213, 'l2_leaf_reg': 1.4661239718658522e-05, 'bagging_temperature': 0.5589807166930933, 'random_strength': 0.012423989754658941}. Best is trial 0 with value: 0.16442104830261187.
[I 2025-05-06 19:41:43,098] Trial 1 finished with value: 0.1607814933346547 and parameters: {'depth': 8, 'learning_rate': 0.25264699552442194, 'iterations': 103, 'l2_leaf_reg': 0.006291772738557463, 'bagging_temperature': 0.4591177565530571, 'random_strength': 3.225770009537909e-05}. Best is trial 1 with value: 0.1607814933346547.
[I 2025-05-06 19:41:43,565] Trial 2 finished with value: 0.5956041852990945 and parameters: {'depth': 10, 'learning_rate': 0.1420391004229679, 'iterations': 246, 'l2_leaf_reg': 4.1324807636497955e-05, 'bagging_temperature': 0.7905132094008895, 'random_strength': 63.90388445848023}. Best is trial 1 with value: 0.1607814933346547.


In [ ]:
import joblib
from ARISA_DSML.train import train_cv
params = joblib.load(best_params_path)
display(params)
n_folds = 5
cv_output_path = train_cv(X_train, y_train, params, n=n_folds)


{'depth': 5,
 'learning_rate': 0.2995567111497413,
 'iterations': 124,
 'l2_leaf_reg': 0.11084080958223169,
 'bagging_temperature': 0.9921093034173206,
 'random_strength': 0.19224056696382372}

Training on fold [0/5]


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.9890110	test: 0.8000000	best: 0.8000000 (0)	total: 1.9ms	remaining: 234ms
1:	learn: 1.0000000	test: 0.8000000	best: 0.8000000 (0)	total: 2.62ms	remaining: 160ms
2:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (2)	total: 5.32ms	remaining: 215ms
3:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (2)	total: 6.95ms	remaining: 208ms
4:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (2)	total: 9.7ms	remaining: 231ms
5:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (2)	total: 12.8ms	remaining: 252ms
6:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (2)	total: 14.8ms	remaining: 248ms
7:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (2)	total: 16.8ms	remaining: 243ms
8:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (2)	total: 18.5ms	remaining: 237ms
9:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (2)	total: 19.5ms	remaining: 223ms
10:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (2)	total: 22.2ms	remaining: 228ms
11:	learn: 1.0000000	test: 1.0000000	best: 1

In [ ]:
# Importing scikit-learn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold, cross_val_score 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import SVC

In [114]:
# Checking if the file exists and the directory
# os.listdir()

In [125]:
# Defining model and hyperparameters for Logistic Regression
model_lr = LogisticRegression(max_iter=1000)
param_grid_lr = {
    'C': [0.1, 1, 10],            
    'penalty': ['l1', 'l2'],       
    'solver': ['liblinear', 'saga']  
}

cv = RepeatedStratifiedKFold(
    n_splits=5,       # Number of folds per repetition
    n_repeats=10,     # Number of repetitions
    random_state=42   # Seed for reproducibility
)

# GridSearchCV configuration
grid_lr = GridSearchCV(
    estimator=model_lr,
    param_grid=param_grid_lr,
    cv=cv,                          
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# Fit for training
grid_lr.fit(X_train, y_train)

#Accuracy (trening)
print("Logistic Regression - Best parameters", grid_lr.best_params_)
print("Logistic Regression - CV Accuracy (trening):", grid_lr.best_score_)

grid_lr_best_model = grid_lr.best_estimator_
cv_scores_grid_lr = cross_val_score(
    grid_lr_best_model,
    X_train,
    y_train,
    cv=cv,            # Same cross-validation strategy
    scoring='accuracy',
    n_jobs=-1
)

print("\nCross-Validation Results (RepeatedStratifiedKFold):")
print(f"Mean Accuracy: {cv_scores_grid_lr.mean():.3f}")
print(f"Standard Deviation: {cv_scores_grid_lr.std():.3f}")

# Test accuracy
y_pred_lr = grid_lr_best_model.predict(X_test)
print("Logistic Regression - Accuracy (validation):", accuracy_score(y_test, y_pred_lr))
print("Logistic Regression - F1 (validation):", f1_score(y_test, y_pred_lr))


Fitting 50 folds for each of 12 candidates, totalling 600 fits
Logistic Regression - Best parameters {'C': 0.1, 'penalty': 'l2', 'solver': 'saga'}
Logistic Regression - CV Accuracy (trening): 0.7325

Cross-Validation Results (RepeatedStratifiedKFold):
Mean Accuracy: 0.733
Standard Deviation: 0.095
Logistic Regression - Accuracy (validation): 0.6
Logistic Regression - F1 (validation): 0.6


In [126]:
# Defining model and hyperparameters for SVM with  RBF
model_svm = SVC(kernel='rbf')
param_grid_svm = {
    'C': [0.01, 0.1, 1, 10, 100, 1000],       
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1, 10],
    'class_weight': [None, 'balanced']
}

cv = RepeatedStratifiedKFold(
    n_splits=10,      # Liczba foldów w każdej iteracji
    n_repeats=3,      # Liczba powtórzeń walidacji krzyżowej
    random_state=42   # Ziarno losowości dla reprodukowalności
)

# GridSearchCV config
grid_svm = GridSearchCV(
    estimator=model_svm,
    param_grid=param_grid_svm,
    cv=10,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# Fit for training
grid_svm.fit(X_train, y_train)

# Accuracy (trening)
print("\nSVM - Best parameters:", grid_svm.best_params_)
print("SVM - Accuracy (trening):", grid_svm.best_score_)

grid_svm_best_model = grid_svm.best_estimator_
cv_scores_grid_svm = cross_val_score(
    grid_svm_best_model,
    X_train,
    y_train,
    cv=cv,
    scoring='accuracy',
    n_jobs=-1
)

print("\nCross-Validation Results (RepeatedStratifiedKFold):")
print(f"Mean Accuracy: {cv_scores_grid_svm.mean():.3f}")
print(f"Standard Deviation: {cv_scores_grid_svm.std():.3f}")

# Test accuracy
y_pred_svm = grid_svm_best_model.predict(X_test)
print("SVM - Accuracy (test):", accuracy_score(y_test, y_pred_svm))
print("SVM - F1 (test):", f1_score(y_test, y_pred_svm))


Fitting 10 folds for each of 84 candidates, totalling 840 fits

SVM - Best parameters: {'C': 10, 'class_weight': None, 'gamma': 'scale'}
SVM - Accuracy (trening): 0.8375

Cross-Validation Results (RepeatedStratifiedKFold):
Mean Accuracy: 0.842
Standard Deviation: 0.096
SVM - Accuracy (test): 0.85
SVM - F1 (test): 0.7692307692307693


In [127]:
# Load data from the Excel file
df_predict_cholesterol = pd.read_excel('predict_cholesterol.xlsx', sheet_name='Sheet1')
df_predict_cholesterol = preprocess_categorical_data(df_predict_cholesterol,mappings, one_hot_columns)


In [128]:
X_val = df_predict_cholesterol.drop(columns=['id'])
X_val_scaled, scaler = scale_features(X_val)

In [129]:
# Function to predict using multiple models
def predict_with_models(models, X):
    predictions = {}
    
    for model_name, model in models.items():
        # Generate predictions for each model
        predictions[model_name] = model.predict(X)
    
    return predictions

In [130]:
# Define models
models = {
    'Random Forest Classifier': grid_rf,
    'Logistic Regression' : grid_lr,
    'AVM': grid_svm

}
# Generate predictions for all models
predictions = predict_with_models(models, X_val_scaled)

# Display the predictions
for model_name, preds in predictions.items():
    print(f"{model_name} Predictions:")
    print(preds)

Random Forest Classifier Predictions:
[0 1 1 1 1 1 1 0 0 0 0 1 1 1 0 1 0 0 1 1]
Logistic Regression Predictions:
[0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 1 1]
AVM Predictions:
[0 1 1 1 1 1 1 0 1 0 0 1 1 1 0 1 0 0 0 1]


In [131]:
def write_predictions_to_excel(models, predictions, input_file, output_file):
    # Load the original dataset
    df = pd.read_excel(input_file) # 'predict_cholesterol.xlsx'

    # Add predictions to the DataFrame
    for model_name, preds in predictions.items():
        df[model_name + '_Predictions'] = preds
    # Save the updated DataFrame to a new Excel file
    df.to_excel(output_file, index=False)

In [132]:
# Write predictions to the new Excel with my surname and name
write_predictions_to_excel(models, predictions, 'predict_cholesterol.xlsx', 'predict_cholesterol_borkowska_joanna.xlsx')

print("Predictions written to predict_cholesterol_borkowska_joanna.xlsx")



Predictions written to predict_cholesterol_borkowska_joanna.xlsx
